In [7]:
from rdkit import Chem, RDConfig
from rdkit.Chem import AllChem, MACCSkeys, rdMolDescriptors, ChemicalFeatures
import numpy as np
import pandas as pd
import os
import deepchem
deepchem.__version__
# SMILES 문자열을 분자 객체로 변환
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

분자구조 pca

In [19]:
target_df = pd.read_csv('./data/finger_ecfp6_train.csv').drop(columns=['id','SMILES', 'MLM', 'HLM', 'AlogP', 'Molecular_Weight', 'Num_H_Acceptors',
       'Num_H_Donors', 'Num_RotatableBonds', 'LogD', 'Molecular_PolarSurfaceArea'])
target_df_test =  pd.read_csv('./data/finger_ecfp6_test.csv').drop(columns=['id','SMILES', 'AlogP', 'Molecular_Weight', 'Num_H_Acceptors',
       'Num_H_Donors', 'Num_RotatableBonds', 'LogD', 'Molecular_PolarSurfaceArea'])
target_df_unlabel = pd.read_csv('./data/finger_ecfp6_unlabel.csv').drop(columns=['SMILES','Unnamed: 1'])

merge_df = pd.concat([target_df,target_df_test,target_df_unlabel])

# PCA 변환
pca = PCA(n_components=3)
merge_pca = pca.fit_transform(merge_df)


In [20]:

# K-means 클러스터링
cluster = 7
kmeans = KMeans(n_clusters=cluster) 
clusters = kmeans.fit_predict(merge_pca)
# 실루엣 점수 계산
# score = silhouette_score(merge_pca, clusters)
# print(f"Silhouette Score: {score}")

# # 3차원 시각화
# fig = plt.figure(figsize=(10, 7))
# ax = fig.add_subplot(111, projection='3d')
# scatter = ax.scatter(merge_pca[:, 0],merge_pca[:, 1],merge_pca[:, 2], c=clusters, cmap='rainbow')

# # 제목 및 레이블 설정
# ax.set_title('K-means Clustering after PCA in 3D')
# ax.set_xlabel('PCA 1')
# ax.set_ylabel('PCA 2')
# ax.set_zlabel('PCA 3')
# plt.colorbar(scatter)

# plt.show()

merge_pca = pd.DataFrame(merge_pca)
merge_pca['cluster'] = clusters

target_df = pd.read_csv('./origin_data/train.csv').drop(columns=['id'])
target_df_test =  pd.read_csv('./origin_data/test.csv').drop(columns=['id'])
target_df_unlabel = pd.read_csv('./data/unlabeld_data2.csv')
# 데이터프레임 연결
target_df = pd.concat([target_df, merge_pca.iloc[:len(target_df), :]], axis=1)
target_df_test = pd.concat([target_df_test, merge_pca.iloc[len(target_df):len(target_df)+len(target_df_test), :].reset_index()], axis=1)
target_df_unlabel = pd.concat([target_df_unlabel, merge_pca.iloc[len(target_df)+len(target_df_test):, :].reset_index()], axis=1)



c:\ProgramData\anaconda3\envs\deepchem\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [ ]:
target_df.to_csv('./data/train_pca.csv',index=False)
target_df_test.drop(columns=['index'],inplace=True)
target_df_test.to_csv('./data/test_pca.csv',index=False)
target_df_unlabel.drop(columns=['index'],inplace=True)
target_df_unlabel.to_csv('./data/unlabeled_pca.csv',index=False)

Atom-pair Fingerprints

In [5]:
def atom_pair(df):
    fingerprint_list = []
    
    for smile in df['SMILES']:
        mol = Chem.MolFromSmiles(f"{smile}")

        # 분자 객체에서 ECFP 지문 생성
        fp = rdMolDescriptors.GetHashedAtomPairFingerprintAsBitVect(mol)
        # 지문을 배열로 변환
        arr = np.array(fp)
        fingerprint_list.append(arr)

    fingerprint_df = pd.DataFrame(fingerprint_list)

    
    return fingerprint_df

name = "atom_pair"

train_df = pd.read_csv('./origin_data/train.csv')

finger_df = atom_pair(train_df)

finger_train_df = pd.concat([train_df,finger_df],axis=1)

finger_train_df.to_csv(f'./data/finger_{name}_train.csv')

test_df = pd.read_csv('./origin_data/test.csv')

finger_df = atom_pair(test_df)

finger_test_df = pd.concat([test_df,finger_df],axis=1)

finger_test_df.to_csv(f'./data/finger_{name}_test.csv')

Substructure/Path-based Fingerprints

In [7]:
def Topological(df):
    fingerprint_list = []
    
    for smile in df['SMILES']:
        mol = Chem.MolFromSmiles(f"{smile}")

        
        fp =  AllChem.RDKFingerprint(mol)

        # 지문을 배열로 변환
        arr = np.array(fp)
        fingerprint_list.append(arr)

    fingerprint_df = pd.DataFrame(fingerprint_list)

    
    return fingerprint_df

name = "topological"

train_df = pd.read_csv('./origin_data/train.csv')

finger_df = Topological(train_df)

finger_train_df = pd.concat([train_df,finger_df],axis=1)

finger_train_df.to_csv(f'./data/finger_{name}_train.csv')

test_df = pd.read_csv('./origin_data/test.csv')

finger_df = Topological(test_df)

finger_test_df = pd.concat([test_df,finger_df],axis=1)

finger_test_df.to_csv(f'./data/finger_{name}_test.csv')

ecfp

In [6]:
def finger_ecfp(df):
    fingerprint_list = []
    
    for smile in df['SMILES']:
        mol = Chem.MolFromSmiles(f"{smile}")

        # 분자 객체에서 ECFP 지문 생성
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=6, nBits=2048)
        # 지문을 배열로 변환
        arr = np.array(fp)
        fingerprint_list.append(arr)

    fingerprint_df = pd.DataFrame(fingerprint_list)

    
    return fingerprint_df

name = "ecfp6"

train_df = pd.read_csv('./origin_data/train.csv')

finger_df = finger_ecfp(train_df)

finger_train_df = pd.concat([train_df,finger_df],axis=1)

finger_train_df.to_csv(f'./data/finger_{name}_train.csv',index = False)

test_df = pd.read_csv('./origin_data/test.csv')

finger_df = finger_ecfp(test_df)

finger_test_df = pd.concat([test_df,finger_df],axis=1)

finger_test_df.to_csv(f'./data/finger_{name}_test.csv',index = False)

unlabel = pd.read_csv('./hiv_SMILES.csv')

finger_df = finger_ecfp(unlabel)

finger_unlabel_df = pd.concat([unlabel,finger_df],axis=1)

finger_unlabel_df.to_csv(f'./data/finger_{name}_unlabel.csv',index = False)

[16:53:00] WARNING: not removing hydrogen atom without neighbors
[16:53:00] WARNING: not removing hydrogen atom without neighbors


maccs

In [1]:
def maccs_fp(df):
    fingerprint_list = []
    
    for smile in df['SMILES']:
        mol = Chem.MolFromSmiles(f"{smile}")

        # Torsion Fingerprints
        fp = MACCSkeys.GenMACCSKeys(mol)
        # 지문을 배열로 변환
        arr = np.array(fp)
        fingerprint_list.append(arr)

    fingerprint_df = pd.DataFrame(fingerprint_list)

    
    return fingerprint_df

name = "maccs_fp"

train_df = pd.read_csv('./origin_data/train.csv')
test_df = pd.read_csv('./origin_data/test.csv')

finger_df = maccs_fp(train_df)
finger_train_df = pd.concat([train_df,finger_df],axis=1)

finger_df = maccs_fp(test_df)
finger_test_df = pd.concat([test_df,finger_df],axis=1)

finger_train_df.to_csv(f'./data/finger_{name}_train.csv',index = False)
finger_test_df.to_csv(f'./data/finger_{name}_test.csv',index = False)

Pharmacophore 

In [12]:
# 약동학적 특성 팩토리를 초기화합니다.
fdefName = os.path.join(RDConfig.RDDataDir, 'BaseFeatures.fdef')
factory = ChemicalFeatures.BuildFeatureFactory(fdefName)

def generate_pharmacophore_fingerprint(mol, factory):
    # 분자의 약동학적 특성을 계산합니다.
    feats = factory.GetFeaturesForMol(mol)
    
    # 각 약동학적 특성의 유형을 기반으로 지문을 생성합니다.
    feature_families = factory.GetFeatureFamilies()
    fp = [0] * len(feature_families)
    for feat in feats:
        idx = feature_families.index(feat.GetFamily())
        fp[idx] = 1
        
    return fp


def Pharmacophore_fp(df):
    fingerprint_list = []
    
    for smile in df['SMILES']:
        mol = Chem.MolFromSmiles(f"{smile}")

        # Torsion Fingerprints
        fp = generate_pharmacophore_fingerprint(mol, factory)
        # 지문을 배열로 변환
        arr = np.array(fp)
        fingerprint_list.append(arr)

    fingerprint_df = pd.DataFrame(fingerprint_list)

    
    return fingerprint_df

name = "Pharmacophore_fp"

train_df = pd.read_csv('./origin_data/train.csv')
test_df = pd.read_csv('./origin_data/test.csv')

finger_df = Pharmacophore_fp(train_df)
finger_train_df = pd.concat([train_df,finger_df],axis=1)

finger_df = Pharmacophore_fp(test_df)
finger_test_df = pd.concat([test_df,finger_df],axis=1)

finger_train_df.to_csv(f'./data/finger_{name}_train.csv')
finger_test_df.to_csv(f'./data/finger_{name}_test.csv')

토크나이저


In [1]:
from transformers import AutoModelForMaskedLM, AutoTokenizer, pipeline, RobertaModel, RobertaTokenizer
from bertviz import head_view

model = AutoModelForMaskedLM.from_pretrained("seyonec/PubChem10M_SMILES_BPE_450k")
tokenizer = AutoTokenizer.from_pretrained("seyonec/PubChem10M_SMILES_BPE_450k")

fill_mask = pipeline('fill-mask', model=model, tokenizer=tokenizer)

c:\ProgramData\anaconda3\envs\deepchem\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at seyonec/PubChem10M_SMILES_BPE_450k were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
def call_html():
    import IPython
    display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              "d3": "https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.8/d3.min",
              jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
            },
          });
        </script>
        <style>
            body {
                color: white;
            }
        </style>
        '''))

from transformers import RobertaModel, RobertaTokenizer
from bertviz import head_view

model_version = 'seyonec/PubChem10M_SMILES_BPE_450k'
model = RobertaModel.from_pretrained(model_version, output_attentions=True)  # RobertaModel을 불러옴
tokenizer = RobertaTokenizer.from_pretrained(model_version)  # RobertaTokenizer를 불러옴



sentence_a = "Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1"
inputs = tokenizer.encode_plus(sentence_a, return_tensors='pt', add_special_tokens=True)  # 문장을 토큰화하고 텐서로 변환
input_ids = inputs['input_ids']
attention = model(input_ids)[-1]  # 어텐션 정보를 얻기 위해 모델에 입력하고 마지막 출력을 가져옴
input_id_list = input_ids[0].tolist()  # 토큰 ID 리스트로 변환
tokens = tokenizer.convert_ids_to_tokens(input_id_list)  # 토큰 ID를 토큰으로 변환

call_html()  # HTML 파일을 호출하는 함수

head_view(attention, tokens)  # 어텐션 시각화 함수를 호출하여 어텐션을 시각화

<IPython.core.display.Javascript object>

In [7]:
import deepchem as dc
from rdkit import Chem

generator = dc.utils.ConformerGenerator(max_conformers=5)
propane_mol = generator.generate_conformers(Chem.MolFromSmiles('CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC'))
print("Number of available conformers for propane: ", len(propane_mol.GetConformers()))

coulomb_mat = dc.feat.CoulombMatrix(max_atoms=50)
features = coulomb_mat(propane_mol)
print(features)

KeyboardInterrupt: 